## Retrieving the Dataset
1. Change runtime type to `Python 3 with GPU`
2. Authorize the notebook to mount your Google Drive
3. Decompress the Dataset onto the local disk (Ephemeral Storage of the provisioned Google Colab VM)

In [ ]:
import os
import time
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
start = time.time()
!cat / content/gdrive/My\ Drive/DashGC/20bn-jester-v1 -?? | tar - zx
stop = time.time()
print('Decompression took', round(((stop - start) / 60), 2), 'mins')

In [ ]:
print(os.getcwd())
dirs = os.listdir('20bn-jester-v1')
print(len(dirs))

directories = {dir: len(os.listdir('20bn-jester-v1/' + dir))
               for dir in dirs}
print(len(directories.keys()))
print(min(directories.values()), max(directories.values()))

In [ ]:
def copier(filedir, lastfile):
    '''
    Copy the image multiple times.
    filedir: path to directory
    lastfile: file to duplicate within the filedir directory
    '''

    with open('20bn-jester-v1/' + filedir + '/' + lastfile, 'rb') as mainfile:
        data = mainfile.read()
    last = int(lastfile[0:-4])
    while last < 70:
        last += 1
        new = str(last)
        fname = ('0' * (5 - len(new))) + new + '.jpg'
        with open('20bn-jester-v1/' + filedir + '/' + fname, 'wb') as file:
            file.write(data)

In [ ]:
tic = time.time()
for dir in dirs:
    count = len(os.listdir('20bn-jester-v1/' + dir))
    chars = 1 if count < 10 else 2
    lastfile = '0' * (5 - chars) + str(count) + '.jpg'
    copier(dir, lastfile)
toc = time.time()
print('Created duplicates in', round(((toc - tic) / 60), 2), 'mins')